<a href="https://colab.research.google.com/github/AvinashReddy3108/Torrent-To-Google-Drive-Downloader/blob/main/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent -> Google Drive uploader
*(ab)using QBitTorrent's WebUI and ngrok.*

Made for personal use with [this awesome guide](https://medium.com/@martinetmayank/create-a-torrent-server-from-google-colab-8fac019bdce2) as base!

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Mounting Google Drive
*DUH, Where else would you store your files?*

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Installing QBittorrent (+ the Web API client)


In [ ]:
!add-apt-repository ppa:qbittorrent-team/qbittorrent-stable -y
!apt install qbittorrent-nox
!pip install qbittorrent-api
!git clone --single-branch --branch latest-release https://github.com/WDaan/VueTorrent.git

### Installing and authenticating [ngrok](https://ngrok.com/)



Get your free auth token at [ngrok's dashboard](https://dashboard.ngrok.com/auth/your-authtoken)!



In [ ]:
# These are some free tokens, uncomment any of them to try them out...

# TOKEN = "1X7aYWPuFKYzvewLbnNoMo71kZi_2uzbB966Q4TU5cpgNPKhy"
# TOKEN = "1UqHsShi6o3ketf426P5UtVdTfs_5XFD6sFRMkryka8fAbLd3"
# TOKEN = "1Qe1IeySOQWSTnpQ3eFfr8j7Oi5_2zhanqnpZwHBhsfANd6yf"
# TOKEN = "7pWLVhS1gxiMAQdaFeYJy_31krnw9drNLLJftaNSFnm"

# Or, be the better guy and use your own token..
TOKEN = "<your ngrok token here>"

def install_ngrok():
    import os
    from zipfile import ZipFile
    from urllib.request import urlretrieve
    
    url = 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip'
    urlretrieve(url, 'ngrok-amd64.zip')
    
    with ZipFile('ngrok-amd64.zip', 'r') as zip_ref:
        zip_ref.extractall('/usr/local/bin/')
    os.chmod('/usr/local/bin/ngrok', 0o755)
    os.unlink('ngrok-amd64.zip')

install_ngrok()
if TOKEN != "":
    !ngrok authtoken $TOKEN

### Start the Qbittorent daemon.

In [5]:
!qbittorrent-nox -d --webui-port=9999

### Custom configuration.

- Ability to override WebUI username/password.
- Using custom [WebUI theme](https://github.com/WDaan/VueTorrent).
- Default location: `your_drive/Torrents`
- Anonymous mode: ON

In [ ]:
import json
import qbittorrentapi

# WebUI Authentication.
_username = "vuetorrent"
_password = "password"

# DO NOT TOUCH THIS, AT ALL!
# --------------------------
try:
  qbt_client = qbittorrentapi.Client(
      host='localhost:9999',
      username='admin',
      password='adminadmin',
      SIMPLE_RESPONSES=True
      )
  prefs = qbt_client.app.preferences
except qbittorrentapi.exceptions.LoginFailed:
  qbt_client = qbittorrentapi.Client(
      host='localhost:9999',
      username=_username,
      password=_password,
      SIMPLE_RESPONSES=True
      )
  prefs = qbt_client.app.preferences
# --------------------------

# Set your prefs here!
prefs['web_ui_username'] = _username
prefs['web_ui_password'] = _password
prefs['alternative_webui_enabled'] = True
prefs['alternative_webui_path'] = 'VueTorrent'
prefs['save_path'] = '/content/drive/MyDrive/Torrents'
prefs['anonymous_mode'] = True
# End of custom prefs.

qbt_client.app.preferences = prefs
print(f'qBittorrent: {qbt_client.app.version} [Web API: {qbt_client.app.web_api_version}]')
print()
print("[Preferences / Configuration]")
print()
print(json.dumps(qbt_client.app.preferences, indent=4, sort_keys=True))

### Start the damn server!

In [ ]:
import threading
import time
import requests
import json
import subprocess

def ngrok(port):
    
    ngrok_cmd = subprocess.Popen(['ngrok', 'http', str(port)])    
    localhost_url = "http://localhost:4040/api/tunnels"

    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text
    json_data = json.loads(tunnel_url)

    tunnel_url = json_data['tunnels'][0]['public_url']
    tunnel_url = tunnel_url.replace("https", "http")
    print('Running at localhost:' + str(port))
    print(tunnel_url)

if __name__ == '__main__':
    
    port = 9999
    thread_ngrok = threading.Thread(target = ngrok, args=(int(port),))

    time.sleep(5)
    print('Establishing secure connection!')
    
    thread_ngrok.start()
    print('Secure connection established...')
    print(f'Username: {_username if _username != "" else "admin"}')
    print(f'password: {_password if _password !="" else "adminadmin"}')
    
    thread_ngrok.join()

### Keep-alive (run and forget.)
*It does what it says :)*

**NOTE: Don't close this window/tab!**

In [ ]:
while True:pass